In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Paths
data_dir = '/kaggle/input/ecg-analysis/ECG_DATA'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Image generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Improved CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Optional: Early stopping and best model saving
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_causal_ecg_model.keras', monitor='val_loss', save_best_only=True)
]


# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=20,
    callbacks=callbacks
)

# Evaluate
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Save final model
model.save('final_causal_ecg_model_1.keras')

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.2976 - loss: 13.6663 - val_accuracy: 0.1853 - val_loss: 14.9885
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


95/95 ━━━━━━━━━━━━━━━━━━━━ 94s 934ms/step - accuracy: 0.3164 - loss: 1.3428 - val_accuracy: 0.1853 - val_loss: 22.7852
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 152us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 91s 906ms/step - accuracy: 0.3540 - loss: 1.3144 - val_accuracy: 0.1853 - val_loss: 22.7153
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 160us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 98s 941ms/step - accuracy: 0.3858 - loss: 1.2855 - val_accuracy: 0.2974 - val_loss: 19.3720
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 96s 955ms/step - accuracy: 0.3847 - loss: 1.2396 - val_accuracy: 0.3265 - val_loss: 10.4402
Epoch 10/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 90s 891ms/step - accuracy: 0.4283 - loss: 1.2256 - val_accuracy: 0.2640 - v

## using only causal feature map and data

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
    Lambda, concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Paths
data_dir = '/kaggle/input/ecg-analysis/ECG_DATA'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# === Causality Map Layer ===
def compute_causality_map(feature_maps):
    def causal_layer(x):
        # x shape: (batch, h, w, channels)
        x_transposed = tf.transpose(x, perm=[0, 3, 1, 2])  # (batch, channels, h, w)
        x_flat = tf.reshape(x_transposed, (tf.shape(x)[0], tf.shape(x)[3], -1))  # (batch, channels, h*w)
        causality = tf.matmul(x_flat, x_flat, transpose_b=True)  # (batch, channels, channels)
        norm = tf.norm(causality, axis=[1, 2], keepdims=True) + 1e-8
        causality_normalized = causality / norm
        return causality_normalized
    return Lambda(causal_layer)(feature_maps)

# === Build CA-CNN Model using Functional API ===
def build_ca_cnn(input_shape=(150, 150, 3), num_classes=4):
    inputs = Input(shape=input_shape)

    # Convolutional Blocks
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Save feature maps before flattening
    feature_maps = x

    # Compute causality map
    causality_map = compute_causality_map(feature_maps)
    causality_flat = Flatten()(causality_map)

    # Flatten feature maps
    features_flat = Flatten()(feature_maps)

    # Concatenate features and causal info
    combined = concatenate([features_flat, causality_flat])

    # Dense layers
    x = Dense(256, activation='relu')(combined)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build the model
model = build_ca_cnn(input_shape=(150, 150, 3), num_classes=4)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_causal_ecg_model.keras', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=20,
    callbacks=callbacks
)

# Evaluate
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Save final model
model.save('final_causal_ecg_model_2.keras')

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 103s 954ms/step - accuracy: 0.3338 - loss: 11.5466 - val_accuracy: 0.2575 - val_loss: 3.5167
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 89s 888ms/step - accuracy: 0.3872 - loss: 1.2804 - val_accuracy: 0.1853 - val_loss: 32.4761
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 87s 869ms/step - accuracy: 0.4385 - loss: 1.2224 - val_accuracy: 0.1853 - val_loss: 50.7218
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 146us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 89s 888ms/step - accuracy: 0.4658 - loss: 1.1815 - val_acc

## using only causal feature map

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
    Lambda, concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Paths
data_dir = '/kaggle/input/ecg-analysis/ECG_DATA'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# === Causality Map Layer ===
def compute_causality_map(feature_maps):
    def causal_layer(x):
        # x shape: (batch, h, w, channels)
        x_transposed = tf.transpose(x, perm=[0, 3, 1, 2])  # (batch, channels, h, w)
        x_flat = tf.reshape(x_transposed, (tf.shape(x)[0], tf.shape(x)[3], -1))  # (batch, channels, h*w)
        causality = tf.matmul(x_flat, x_flat, transpose_b=True)  # (batch, channels, channels)
        norm = tf.norm(causality, axis=[1, 2], keepdims=True) + 1e-8
        causality_normalized = causality / norm
        return causality_normalized
    return Lambda(causal_layer)(feature_maps)

# === Build CA-CNN Model using Functional API ===
def build_ca_cnn(input_shape=(150, 150, 3), num_classes=4):
    inputs = Input(shape=input_shape)

    # Convolutional Blocks
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Save feature maps before flattening
    feature_maps = x

    # Compute causality map
    causality_map = compute_causality_map(feature_maps)
    causality_flat = Flatten()(causality_map)

    # Flatten feature maps
    features_flat = Flatten()(feature_maps)

    # Concatenate features and causal info
    combined = causality_flat

    # Dense layers
    x = Dense(256, activation='relu')(combined)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build the model
model = build_ca_cnn(input_shape=(150, 150, 3), num_classes=4)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_causal_ecg_model.keras', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=20,
    callbacks=callbacks
)

# Evaluate
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Save final model
model.save('final_causal_ecg_model_3.keras')


GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 103s 956ms/step - accuracy: 0.3401 - loss: 1.3273 - val_accuracy: 0.2511 - val_loss: 1.3886
Epoch 2/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 86s 849ms/step - accuracy: 0.4700 - loss: 1.1222 - val_accuracy: 0.2511 - val_loss: 1.4314
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 147us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 87s 859ms/step - accuracy: 0.5316 - loss: 1.0645 - val_accuracy: 0.2586 - val_loss: 1.6727
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 157us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 90s 896ms/step - accuracy: 0.5653 - loss: 1.0180 - val_accura